In [1]:
import json

In [2]:
json_nsf = json.load(open("../outputs/nsf/nsf_mapping.json"))

In [28]:
with open("../outputs/nsf/nsf_doi_project_summary.tsv", "w", encoding='utf-8') as fo:
    for k in json_nsf.keys():
        for i in json_nsf[k]:
            if i.get('cr_meta'):
                if type(i['cr_meta']['title']) is list:
                    try:
                        title = i['cr_meta']['title'][0]
                    except:
                        title = i['cr_meta']['title']
                else:
                    title = i['cr_meta']['title']            
            else:
                title = '--'
                
            if i.get('cites'):
                fo.write("{}\t{}\t{}\t{}\t{}\t{}\n".format(
                  k, i['doi'].lower(), 
                  title.replace("\n", ""), 
                  i['ams_bib'].replace("\n", ""), i['cites'], i['cr_meta']['issued']['date-parts'][0][0])
                 )
            else:
                if type(title) is list:
                    title = ""
                fo.write("{}\t{}\t{}\t--\t--\t--\n".format(
                  k, i['doi'].lower().strip(), title.replace("\n", ""), 
                  )
                 )

In [29]:
import pandas as pd

In [30]:
df = pd.read_csv("../outputs/nsf/nsf_doi_project_summary.tsv", sep='\t', header=None)
df.columns = ['nsfid', 'doi', 'title', 'ams_bib', 'cites', 'year']

df_clean = df.query('doi != ""').drop_duplicates()

In [31]:
df_clean.head()

,nsfid,doi,title,ams_bib,cites,year
0,1324760,10.1016/j.geomorph.2015.03.039,"Data management, sharing, and reuse in experim...","Hsu, L., R. L. Martin, B. McElroy, K. Litwin-M...",19,2015
2,1324760,10.2110/sedred.2013.4.9,Building a Sediment Experimentalist Network (S...,"Hsu, L., B. McElroy, R. L. Martin, and W. Kim,...",2,2013
3,1324760,10.2110/sedred.2013.4,NaN,--,--,--
4,1340233,10.1126/science.342.6162.1041-b,Open Data: Crediting a Culture of Cooperation,"Bolukbasi, B., and Coauthors, 2013: Open Data:...",10,2013
5,1340233,10.1002/2014eo200001,Community-Developed Geoscience Cyberinfrastruc...,"Richard, S. M., G. Pearthree, A. K. Aufdenkamp...",7,2014


In [32]:
df_clean.query('title=="--"')[['doi', 'nsfid']]

,doi,nsfid
6,NaN,1340265
12,NaN,1343760
26,NaN,1343800
32,NaN,1343802
33,10.1002/hyp10899,1343802
41,NaN,1343811
56,NaN,1343813
60,NaN,1440081
62,NaN,1440109
65,NaN,1440139


In [33]:
df_clean.doi.value_counts()

10.1002/gdj3.36                        4
10.1002/2016ea000237                   4
10.1017/s1743921317000060              3
10.1016/j.compenvurbsys.2016.10.010    3
10.3233/sw-160216                      3
                                      ..
10.1017/qua.2017.105                   1
10.22498/pages.26.2.78                 1
10.21425/f5fbg50711                    1
10.22498/pages.26.2.64                 1
10.1128/mbio.01318-19                  1
Name: doi, Length: 231, dtype: int64

In [34]:
groups = df_clean.groupby('doi').groups
for g in groups.keys():
    nsf_projects = list(set([d['nsfid'] for i, d in df_clean.loc[groups[g]].iterrows()]))
    title, year, cites = df_clean.query(f'doi == "{g}"')[['title','year', 'cites']].values[0]
    print(title, g, year, cites, f"({','.join([str(p) for p in nsf_projects])})")

        # print(df_clean.query(f"doi == '{g}'")['nsfid'])

Community-Developed Geoscience Cyberinfrastructure 10.1002/2014eo200001 2014 7 (1340233)
Science and Cyberinfrastructure: The Chicken and Egg Problem 10.1002/2014eo490006 2014 3 (1343811)
Toward the Geoscience Paper of the Future: Best practices for documenting and sharing research from data to software to provenance 10.1002/2015ea000136 2016 94 (1440323)
Hydrocomplexity: Addressing water security and emergent environmental risks 10.1002/2015wr017342 2015 35 (1440315)
An introduction to the special issue on Geoscience Papers of the Future 10.1002/2016ea000201 2016 10 (1440332)
Reproducible, component‐based modeling with TopoFlow, a spatial hydrologic modeling toolkit 10.1002/2016ea000237 2017 2 (1639547,1343811,1440332,1440333)
BedMachine v3: Complete Bed Topography and Ocean Bathymetry Mapping of Greenland From Multibeam Echo Sounding Combined With Mass Conservation 10.1002/2017gl074954 2017 356 (1541390)
Ushering in a New Frontier in Geospace Through Data Science 10.1002/2017ja024835